In [1]:
import Python

In [2]:
let np = Python.import("numpy")
let plt = Python.import("matplotlib.pyplot")
let torch = Python.import("torch")
let fastai = Python.import("fastai")
let fastaiVision = Python.import("fastai.vision")
let fastaiMetrics = Python.import("fastai.metrics")

2019-04-19 12:39:16.377745: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0


In [3]:
let bs = 64

In [4]:
%include "EnableIPythonDisplay.swift"

In [5]:
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


In [6]:
Python.help(fastaiVision.untar_data)

Help on function untar_data in module fastai.datasets:

uunnttaarr__ddaattaa(url:str, fname:Union[pathlib.Path, str]=None, dest:Union[pathlib.Path, str]=None, data=True, force_download=False) -> pathlib.Path
    Download `url` to `fname` if it doesn't exist, and un-tgz to folder `dest`.



None


In [7]:
let path = fastaiVision.untar_data(fastaiVision.URLs.PETS);path

/home/omar/.fastai/data/oxford-iiit-pet


In [8]:
path.ls()

[PosixPath('/home/omar/.fastai/data/oxford-iiit-pet/images'), PosixPath('/home/omar/.fastai/data/oxford-iiit-pet/annotations')]


In [9]:
let path_anno = path/"annotations"
let path_img = path/"images"

In [10]:
let fnames = fastaiVision.get_image_files(path_img)
fnames[0 ..< 5]

[PosixPath('/home/omar/.fastai/data/oxford-iiit-pet/images/British_Shorthair_136.jpg'), PosixPath('/home/omar/.fastai/data/oxford-iiit-pet/images/japanese_chin_72.jpg'), PosixPath('/home/omar/.fastai/data/oxford-iiit-pet/images/shiba_inu_144.jpg'), PosixPath('/home/omar/.fastai/data/oxford-iiit-pet/images/samoyed_72.jpg'), PosixPath('/home/omar/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_133.jpg')]


In [11]:
np.random.seed(2)
let pat = "/([^/]+)_\\d+.jpg$"

In [12]:
var data = fastaiVision.ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms: fastaiVision.get_transforms(), size: 224, bs: bs)

In [13]:
data = data.normalize(fastaiVision.imagenet_stats)

In [14]:
//data.show_batch(rows: 3)//, figsize: (7,6))

In [15]:
print(data.classes,Python.len(data.classes),data.c)

['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx', 'american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'boxer', 'chihuahua', 'english_cocker_spaniel', 'english_setter', 'german_shorthaired', 'great_pyrenees', 'havanese', 'japanese_chin', 'keeshond', 'leonberger', 'miniature_pinscher', 'newfoundland', 'pomeranian', 'pug', 'saint_bernard', 'samoyed', 'scottish_terrier', 'shiba_inu', 'staffordshire_bull_terrier', 'wheaten_terrier', 'yorkshire_terrier'] 37 37


In [16]:
var learn = fastaiVision.cnn_learner(data, fastaiVision.models.resnet34, metrics: fastaiVision.error_rate)

In [17]:
learn.model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [18]:
learn.fit_one_cycle(4)

epoch     train_loss  valid_loss  error_rate  time    
2019-04-19 12:39:25.715277: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
2019-04-19 12:39:26.901118: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
0         1.417381    0.311485    0.088633    00:16     
1         0.567237    0.234128    0.066982    00:14     
2         0.328307    0.205842    0.065629    00:14     
3         0.244856    0.197086    0.056834    00:14     


None


In [19]:
learn.save("stage-1")

None


In [20]:
var interp = fastaiVision.ClassificationInterpretation.from_learner(learn)

In [21]:
var x = interp.top_losses()
var losses=x[0]
var idxs=x[1]

In [22]:
losses

tensor([8.4222e+00, 8.1818e+00, 7.2055e+00,  ..., 1.9073e-06, 1.9073e-06,
        -0.0000e+00])


In [23]:
idxs

tensor([1305,  743,   87,  ...,  924,  532,   50])


In [24]:
print(Python.len(data.valid_ds),Python.len(losses),Python.len(idxs))

1478 1478 1478


In [25]:
//interp.plot_top_losses(9)

In [26]:
//interp.plot_confusion_matrix( dpi: 60)

In [27]:
interp.most_confused(min_val: 2)

[('staffordshire_bull_terrier', 'american_pit_bull_terrier', 5), ('Ragdoll', 'Birman', 4), ('Russian_Blue', 'British_Shorthair', 4), ('american_pit_bull_terrier', 'staffordshire_bull_terrier', 4), ('boxer', 'american_bulldog', 4), ('Egyptian_Mau', 'Bengal', 3), ('beagle', 'basset_hound', 3), ('miniature_pinscher', 'chihuahua', 3), ('Birman', 'Ragdoll', 2), ('Birman', 'Siamese', 2), ('Maine_Coon', 'Abyssinian', 2), ('Maine_Coon', 'Ragdoll', 2), ('Russian_Blue', 'Bombay', 2), ('Siamese', 'Birman', 2), ('american_bulldog', 'american_pit_bull_terrier', 2), ('american_pit_bull_terrier', 'american_bulldog', 2), ('basset_hound', 'american_pit_bull_terrier', 2), ('basset_hound', 'beagle', 2), ('chihuahua', 'miniature_pinscher', 2)]


In [28]:
learn.unfreeze()

None


In [29]:
learn.fit_one_cycle(1)

epoch     train_loss  valid_loss  error_rate  time    
0         0.537700    0.334759    0.110961    00:16     


None


In [30]:
learn.load("stage-1");

Learner(data=ImageDataBunch;

Train: LabelList (5912 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
British_Shorthair,japanese_chin,shiba_inu,english_setter,keeshond
Path: /home/omar/.fastai/data/oxford-iiit-pet/images;

Valid: LabelList (1478 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
newfoundland,miniature_pinscher,shiba_inu,boxer,Abyssinian
Path: /home/omar/.fastai/data/oxford-iiit-pet/images;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_siz

In [31]:
learn.lr_find()

epoch     train_loss  valid_loss  error_rate  time    
0         0.743794    #na#        00:10         
LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


None


In [32]:
learn.recorder

Recorder
learn: Learner(data=ImageDataBunch;

Train: LabelList (5912 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
British_Shorthair,japanese_chin,shiba_inu,english_setter,keeshond
Path: /home/omar/.fastai/data/oxford-iiit-pet/images;

Valid: LabelList (1478 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
newfoundland,miniature_pinscher,shiba_inu,boxer,Abyssinian
Path: /home/omar/.fastai/data/oxford-iiit-pet/images;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64

In [33]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr: Python.slice(1e-6,1e-4))

epoch     train_loss  valid_loss  error_rate  time    
0         0.237691    0.193621    0.058863    00:16     
1         0.216884    0.190909    0.059540    00:16     


None
